# Emerging Properties in Self-Supervised Vision Transformers.  
- DINO (self-**di**stillation with **no** labels)
- paper review 

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/figure1.png?raw=true">

## Abstract. 
- 우리는 ViT에 SSL을 적용해 CNN에 비해 눈에 띄는 새로운 property를 추출할 수 있을까? 라는 가설을 세운다  
- observation. 
    - 첫 번째로 SSL ViT의 feature는 semantic segmentation에 대한 정보를 담고 있으며 이는 supervised ViT나 CNN에는 드러나지 않는다  
    - 두 번째로 이러한 feature들은 훌륭한 k-NN classifier의 역할도 한다  
- 본 논문에서는 momentum encoder, multi-crop training 그리고 ViT에서 small patch를 사용한다는 것을 강조한다  

## Introduction. 
- ViT는 CNN과 비교해 경쟁력있는 성능을 가지지만 명확한 장점을 가지지 못한다  
- 예를 들어, 계산 복잡도가 증가한다거나 많은 학습 데이터를 필요로 한다거나 등 unique한 property를 보여주지 못하고 있다  
- 우리는 NLP의 Transformer가 self-supervised pretraining의 시도가 성공적이었단 것에 아이디어를 얻었다  
- self-supervised pretraining의 목적은 풍부한 visual information을 가지기 위함이다  
- 반면 supervised의 경우, image마다 label이 하나씩 존재해 단순한 visual information을 가진다  
- NLP에서 사용됐던 self-supervised pretext task는 text 기반이지만 그동안 많은 연구들을 통해 이미지에서도 잠재력을 보여줬다  
- 이들의 구조는 비슷하지만 구성요소들을 조금씩 바꿔가며 성능을 개선시켰다  
- 이 논문에서 우리는 Supervised ViT나 CNN에서 드러나지 않은 property에 대해 연구했다  
    - Self-supervised ViT feature는 scene layout을 명확히 담고 있으며 특히, object boundaries에 대한 정보를 담고 있다  
    - 이러한 정보는 self-attention에서 바로 이해하기 쉽다  
    - Self-supervised ViT feature는 어떤 finetuning, linear classifier 또는 data augemtnation 없이 basic k-NN classifier로도 잘 작동하며 ImageNet에서 top-1 accuracy를 78.3%를 달성했다  
    
- 성능 좋은 k-NN classifier는 momentum encoder와 multi-crop augmentation과 같은 method를 이용할 때만 유효하다  
- 또 다른 중요한 연구결과는 ViT에서 작은 patch를 사용했을 때 feature의 퀄리티가 개선된다는 점이다  
- 종합적으로 이러한 component들의 중요성에 대한 연구결과는 label 없이 knowledge distillation 형태의 간단한 SSL 접근법으로 이끌었다  
- DINO는 standard CE Loss와 momentum encoder로 만들어진 teacher network를 통해 바로 output을 예측함으로써 SSL을 간단하게 했다  
- 흥미롭게도 우리의 method는 collapse를 피하기 위해 teacher output을 centering과 sharpening을 하는 것만으로도 충분했다  
- 반면 이전 연구에서는, advanced normalization, contrastive loss 등 복잡한 method들을 사용했다  
- 우리 framework는 유연하며 architecture 수정이나 internal normalization 적용 없이 ViT와 CNN에 모두 잘 작동된다  
- 우리는 이전 SSL SOTA를 능가함으로써 DINO와 ViT의 시너지를 검증했다  
- 또한 CNN에서도 잘 동작하는 것을 보여준다

## Related work. 
**Self-supervised learning**  
- 우리의 접근법은 BYOL에서의 아이디어를 가져왔지만 다른 점은 조금 다른 matching loss function을 사용하고 student와 teacher의 network를 같은 것을 쓴다는 점이다

**Self-training and knowledge distillation**  
- 이전 논문들에서 teacher network는 학습할 때 고정된 pre-trained weight를 쓰지만 본 논문에서는 학습하도록 설정했다  
- 우리 연구는 student와 teacher가 같은 architecture를 공유하며 훈련하면서 distillation을 사용해 codistillation에 더 가깝다  

## Approach. 
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/figure2.png?raw=true" width=500>

### SSL with Knowledge Distillation. 
- knowledge distillation은 student network $g_{\theta_{s}}$를 주어진 teacher ntwork $g_{\theta_{t}}$의 output과 매칭을 하기 위해 학습하는 방식이다  
- student network의 output은 softmax를 거쳐 다음과 같이 정의된다  
$$P_s\left(x\right)^{(i)}=\frac{exp\left(g_{\theta_s}\left(x\right)^{(i)}/\tau_s\right)}{\sum_{k=1}^Kexp\left(g_{\theta_s}\left(x\right)^{(k)}/\tau_s\right)}$$  
- teacher network의 output도 위와 비슷하다  
- 이때 $\tau_s>0$는 output distribution의 sharpness를 조절하는 temperature parameter이다  
- 고정된 teacher network $g_{\theta_t}$가 주어지면, 우리는 이를 student network $\theta_s$에 대한 cross-entropy loss를 minimizing함으로써 distribution을 매칭하기 위해 학습을 한다  
$$min_{\theta_s}H\left(P_t\left(x\right),P_s\left(s\right)\right)$$  
- 이때 H는 cross-entropy loss. 
- 다음으로, 우리는 위의 식을 가지고 SSL에 어떻게 적용할 것인지 살펴본다  
- 먼저, 우리는 multi-crop의 이미지를 distored views 또는 crops을 적용해 생성한다  
- 더 정확히는 이미지가 주어지면, 우리는 다른 views의 set V를 생성한다  
- 이 set은 두 개의 global views, $x_1^g$, $x_2^g$와 더 작은 resolution의 각 local views를 포함하고 있다  
- student net에는 모든 crops를 통과하며 teacher net에는 global views를 통과한다  
- 따라서 loss function은 다음과 같다  
$$min_{\theta_s}\sum_{x\in\left\{x_1^g,x_2^g\right\}}\sum_{x'\in V, x'\neq}H\left(P_t\left(x\right), P_s\left(x'\right)\right)$$  

- 우리는 stochastic gradient descent를 가지고 위의 loss를 최소화한다  

**Teacher network**  
- knowledge distillation과 다르게 우리 teacher net은 사전 훈련된 모델을 사용하지 않으며 student net의 post iteration의 weight를 사용한다  
- 그리고 teacher을 1 epoch씩 고정하는 것은 잘 작동됐다  
- 반면 student weight를 copy하는 것은 잘 수렴하지 않았다  
- 특히 강조할 점은, student weight에 EMA(exponential moving average)를 사용하는 것이다  
- 즉, momentum encoder를 사용하면 DINOdp wkf akwdkTek. 
- update rule은 $\theta \leftarrow\lambda\theta_t + \left(1-\lambda\right)\theta_s$이며 이때 $\lambda$는 0.996에서 1까지 cosine schedule을 따른다  
- 기존 momentum encoder는 constrative learning에서 queue에 대한 대체재를 사용하지만 우리는 queue가 존재하지 않으며 contrastive loss를 사용하지 않는다  
- 우리 momentum encoder는 mean teacher의 역할에 더 가깝다  
- 우리는 DINO의 teacher가 exponential decay를 사용한 Polyak-Roppert averaging과 비슷한 model ensemble 형태로 수행한다는 것을 관찰했다  
- 이때 Polyak-Roppert averaging은 moel ensemble의 성능을 높이는 일반적인 방법이다  
- 우리는 학습하는 동안 teacher가 student보다 좋은 성능을 내는 것도 확인했다  
- 따라서 high quality의 feature를 student에게 제공해 학습을 guide한다  

**Network architecture**  
- neural network g는 backbone f(ViT or ResNet)와 projection head $h: g=h\circ f$로 구성되어 있다  
- downstream task에서 사용되는 feature는 backbone f의 output이다  
- projection head는 3개의 MLP로 구성되어 있으며 node tnsms 2048rodlrh l2 norm과 weight normalized가 적용됐다   
- 우리는 predictor를 사용하지 않았으며 teacher와 student는 같은 architecture이다   
- 또한 ViT 모델은 Batch Normalization을 사용하지 않는다  

**Avoiding collapse**  
- 각 self-supervised method들은 collapse를 피하기 위해 contrastive loss, clustering constraints, batch normalization 등 많은 방법을 사용했다  
- 반면 DINO는 multiple normalization을 통해 안정화할 수 있지만 collapse를 피하기 위해 momentum teacher output의 centering과 sharpening만으로도 충분했다  
- centering은 one dimension이 지배하는 것을 막아주지만 collapse를 uniform distribution으로 유도하며 sharpening은 반대의 효과를 가진다  
- 따라서 두 연산을 모두 적용해 collapse를 피할 수 있게 균형을 맞춘다  
- 이 방법을 선택하면 batch에 대한 의존도가 낮아진다  
- centering은 first-order batch statistics에만 의존하고 bias term c를 teacher에 더해주는 것으로 해석될 수 있다  
$$g_t\left(x\right)\leftarrow g_t\left(x\right)+c$$  
- center c는 exponential moving average로 업데이트되며 다른 batch size에서도 잘 작동한다  
$$c\leftarrow mc + \left(1-m\right)\frac{1}{B}\sum_{i=1}^Bg_{\theta_t}\left(x_i\right)$$  
- 이때 $m>0$은 rate parameter, B는 batch size  

### Implementation and evaluation protocols. 
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table1.png?raw=true" width=500>

**Vision Transformer**  
- 우리는 DeiT의 구현을 따른다  

**Implementation details**  
- label 없이 ImageNet dataset에서 모델을 사전훈련한다  
- optimizer : AdamW  
- batch size : 1024. 
- learning rate : 처음 10 epoch 동안은 lr = 0.0005 * batchsize/256의 scaling rule을 사용해 lr을 증가시킨 후, cosine schedule을 따른다  
- weight decay : cosine schedule에 따라 0.04에서 0.4까지 사용한다  
- temperature $\tau_s$ : 0.1, linear warm-up 30 epoch 동안에는 0.04에서 0.07까지 증가시킨다  
- data augemtnation : color jittering, gaussian blur, solarization and multi-crop with a bicubic interpolation. 

**Evaluation protocols**  
- SSL에 대한 standard protocol은 frozen feature에서 linear clasifier를 학습하거나 downstream task에서 feature를 finetune하는 것이다  
- linear evaluation에 대해서는 학습하는 동안 random resize crop, horizontal flip augmentation을 적용하고 central crop에서 정확도를 기록한다  
- finetuning evaluation에 대해서는 학습할 때는 pretrained weight를 사용한다  
- 그러나 위의 두 evaluation은 hyperparameter에 굉장히 민감하며 learning rate에 따라 정확도가 다양하게 나타난다  
- 따라서 우리는 simple weighted k-NN classifer로 feature의 quality를 평가한다  
- 우리는 train data의 feature를 계산하고 저장하기 위해 pretrain model을 freeze한다  
- 그런 다음, k-NN classifier는 이미지의 feature를 label에 대해 투표한 k-nearest feature와 매칭시킨다  
- 우리는 20 NN이 일관적으로 잘 작동하는 것을 발견했다   
- 이런 evaluation protocol은 hyperparameter tuning이나 augmentation이 필요하지 않다  

## Main Results  
### Comparing with SSL frameworks on ImageNet.  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table2.png?raw=true" width=450>

**Comparing with the same architecture**  
- 우리는 다른 self-supervised method들과 DINO의 성능 비교를 table 2에서 나타냈다  
- ResNet-50 or ViT-small로 같은 architecture를 사용한다  
    - 비슷한 parameter 수 (21M vs 23M)  
    - throughput (1237/sec vs 1007 im/sec)  
    - performance (79.3% vs 79.8%)  

- 먼저 우리는 DINO가 ResNet-50 SOTA와 동등한 성능을 보여주며 standard setting에서도 잘 작동한다는 것을 관찰했다  
- ViT로 바꿨을 때도 DINO는 BYOL, MoCov2 and SwAV의 성능을 능가한다 (linear classification에서 3.5% 향상, k-NN에서 7.9% 향상)  
- 더 놀라운 것은,간단한  k-NN classifier의 성능이 linear classifier와 동등하다는 것이다  
- 이러한 성질은 ViT의 DINO에서만 나타나며 self-supervised method나 ResNet-50에서는 나타나지 않는다  

**Comparing across architectures**  
- 여기서 목적은 ViT의 사이즈를 더 키우면 그 성능은 어떤지 살펴보는 것이다  
- larger ViT의 DINO는 성능이 향상된 반면, patch size를 더 줄이면 그 효과가 더 크다  
- patch size를 줄이는 것은 파라미터 수에 영향을 미치지 않는 반면 running time이 늘어나고 memory usage가 늘어난다  
- 그래도 base ViT with 8 patches가 top-1에서 linear classification은 80.1%, k-NN은 77.4%로 가장 성능이 좋긴 하다  

### Properties of ViT trained with SSL. 
#### Nearest neighbor retrieval with DINO ViT. 
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table3.png?raw=true" width=450>

**Image Retrieval**  
- Oxford와 Paris dataset을 사용한다  
- table 3에서 supervised와 DINO의 성능을 비교했다  
- feature를 freeze 시키고 바로 k-NN에 적용했다  
- 우리는 DINO가 supervised의 성능을 능가한다는 것을 관찰했다  
- SSL의 장점은 labeling된 data가 필요없다는 점이다  
- GLDv2에서 사전훈련된 DINO는 놀라울 정도로 좋은 성능을 보여준다 

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table4.png?raw=true" width=450>

**Copy detection**  
- Copy detection은 blur, insertion, print, scan 등으로 왜곡된 이미지를 인식하는 task다  
- 우리는 성능 측정을 feature의 cosine similarity로 진행한다  
- table 4에서 DINO의 ViT가 가장 성능이 좋은 것을 알 수 있다  

#### Discovering the semantic layout of scenes. 
- 우리는 standard benchmark에서 나오는 property 뿐만 아니라 attention map에서 생성되는 mask의 quality에 대해서도 살펴볼 것이다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table5.png?raw=true" width=450>

**Video instance segmentation**  
- 이 task는 연속적인 frame에서 nearest neighbor scene를 분할한다  
- 따라서 feature를 가지고 따로 모델을 학습할 필요도 없고 fine-tuning 할 필요도 없다  
- benchmark에서 경쟁력 있는 성능을 보여준다  
- 그리고 $8\times 8$과 같은 작은 patch size를 사용했을 때 가장 좋은 성능을 보여준다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/figure3.png?raw=true" width=450>

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/figure4.png?raw=true" width=450>

**Probing the self-attention map**  
- figure 3에서, 각 head가 이미지에서 서로 다른 semantic region을 처리하는 것을 보여주고 있다  
- 세 번째 행은 object가 다른 object에 의해 가려짐을 보여주고 두 번째 행은 object의 크기가 작은 경우다  
- 위의 두 경우에도 불구하고 semantic mask가 잘 생성되는 것을 볼 수 있다  
- figure 4는 self-supervised가 supervised보다 object의 feature를 clustering하는 데 있어 성능이 더 좋음을 보여준다  
- self-supervised와 supervised의 성능을 Jaccard similarities로 비교하면 그 차이가 더 명확하게 나타난다  
- self-supervised CNN도 segmentation의 정보를 담고 있다  

### Transfer learning on downstream tasks. 
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table6.png?raw=true" width=450>

- downstream task에서도 transfer learning을 할 때 supervision보다 self-supervised의 성능이 더 좋다  

## Ablation Study of DINO. 
### Importance of the Difference Components. 
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table7.png?raw=true" width=450>

- table 7에서 component를 추가하거나 제거해 다양한 모델들의 성능을 기록했다  
- 먼저 momentum이 없으면 아예 동작되지 않음을 살펴봤다  
- 그리고 SK는 collapse를 피하기 위해 필요하다  
- 또한 momentum과 SK를 같이 쓰면 그 효과가 덜 하다는 것도 나타났으며 성능을 위해서는 momentum이 필수적이다  
- 두 번째로 좋은 feature를 얻기 위해서는 multi-crop과 CE Loss가 중요한 component임이 나타났다  
- 그리고 predictor는 추가하지 않는게 더 성능이 좋았다  

<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/figure5.png?raw=true" width=450>

**Importance of the patch size**  
- figure 5에서 16, 8, 5의 patch size로 실험을 한 결과가 나와있다  
- 우리는 patch size가 작을수록 성능이 좋은 것을 확인했다  
- 그러나 patch size가 작을수록 처리 비용이 늘어난다 ($5\times 5$: 44im/s, $8\times 8$: 180im/s)  

### Impact of the choice of Teacher Network  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/figure6.png?raw=true" width=450>

**Building different teachers from the student**  
- student weight는 teacher의 이전 iteration보다 이전 epoch의 weight를 사용하는 것이 더 성능이 좋다  
- 흥미로운 점은 이전 epoch의 teacher를 사용했을 때 collapse가 일어나지 않았다는 점이다  
- 결론적으로 momentum encoder의 teacher를 사용하는 것이 가장 좋은 성능을 낸다  

**Analyzing the training dynamic**  
- momentum teacher가 왜 잘 작동하는지에 대해서 살펴본다  
- 주요 관찰은 teacher가 학습하는 동안 student의 성능을 뛰어넘는다는 것이다  
- 이러한 경향은 ViT뿐만 아니라 ResNet-50에서도 나타났다  
- 이는 우리 method가 Royak-Ruppert averaging으로 인해 학습하면서 끊임없이 우수한 성능의 모델을 ensembling하기 때문이라고 해석될 수 있다  

### Avoiding collapse  
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/figure7.png?raw=true" width=450>  

- collapse는 input에 상관없이 model의 output이 uniform하거나 하나의 dimension이 지배하는 경우를 의미한다  
- centering은 diminant dimension을 막아주지만 uniform한 output을 유도한다  
- shaprening은 그 반대다  
- 둘 중 하나만 사용하면 KL 값이 0으로 수렴하며 collapse가 나타난다  

### Training with small batches. 
<img src = "https://github.com/Sangh0/Self-Supervised-Learning/blob/main/DINO/figure/table9.png?raw=true" width=450>  

- 위의 table에서 batch size가 클수록 성능이 좋아짐을 보여주고 있다  

## Conclusion  
- 본 논문에서 우리는 standard ViT 모델을 self-supervised pretraining의 잠재력에 대해 보여줬고 이는 SOTA CNN 모델보다 좋은 성능을 낸다  
- 또한 k-NN classification에서 feature의 quality가 이미지에 대해 잠재력이 있다는 것도 발견했다  
- scene layout이 있는 feature의 information을 가지고 성능이 좋지 않은 supervised image segmentation에서도 적용해 이점을 얻을 수 있다 
- 본 논문을 통해 self-supervised가 ViT를 기반으로 BERT같은 모델을 개발할 수 있다는 것을 핵심으로 삼는다